### Import modules

In [1]:
from utils import * 

import numpy as np
import pandas as pd
from pprint import pprint
import os
import matplotlib.pyplot as plt
from collections import defaultdict

# Gensim
from gensim.test.utils import datapath
from gensim.test.utils import common_texts, get_tmpfile

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.option_context('display.max_colwidth', 500);

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


config = get_config('config.yaml')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\enlik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\enlik\Miniconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [26]:
df

,userName,review,rating,date
0,Hilary Meyer,Your drivers are great BUT your support is no ...,3,2020-12-06 19:38:13
1,Kenneth B,"Loving the app, but can only give it a medium ...",3,2020-12-11 14:06:15
2,Bonolo Mphahlele,The response time with regards to customer ser...,4,2020-12-07 13:59:18
3,jessfechi egolo,This has been happening for some time now and ...,1,2020-12-13 18:19:06
4,Safeeya Lawal,The drivers almost always never have change an...,3,2020-12-05 15:08:10
...,...,...,...,...
29808,P.Raimond,It’s rather difficult to get a taxi. Sometimes...,3,2018-05-30 05:37:16
29809,sõidan taksoga,Taxi drivers don’t speak Estonian or English. ...,1,2018-11-11 10:32:50
29810,madsp1r1t,Ride booking extension in Apple maps not avail...,3,2019-11-07 18:53:33
29811,hmerith,"I ordered a taxi, but when it arrived the driv...",1,2019-07-12 23:33:57


In [2]:
# setup input and output path
input_path = 'preprocessed_data/all/'
output_path = 'std_lda_vis/all/'

# Import dataset
df = pd.read_csv(config['csv_input_local']['all_reviews_p3'], index_col=0)
df = df.reset_index(drop=True)
reviews = df.review
reviews

0        Your drivers are great BUT your support is no ...
1        Loving the app, but can only give it a medium ...
2        The response time with regards to customer ser...
3        This has been happening for some time now and ...
4        The drivers almost always never have change an...
                               ...                        
29808    It’s rather difficult to get a taxi. Sometimes...
29809    Taxi drivers don’t speak Estonian or English. ...
29810    Ride booking extension in Apple maps not avail...
29811    I ordered a taxi, but when it arrived the driv...
29812    taxis are always late without exception. drive...
Name: review, Length: 29813, dtype: object

NLPpipeline

In [3]:
term_doc = pd.read_pickle(input_path + 'term_doc.pkl')
data_lemmatized = pd.read_pickle(input_path + 'data_lemmatized.pkl')
dictionary = pd.read_pickle(input_path + 'dictionary.pkl')
tf_idf = pd.read_pickle(input_path + 'tf_idf.pkl')

<string>Code for piepelining</strong>
<code style="font-size: 10px; background-color:transparent;">
nlp_pipe = NLPpipe()
term_doc = nlp_pipe.fit_transform(reviews, min_count = 3, threshold = -0.5)
tf_idf = nlp_pipe.transform(reviews, tf_idf = True)
data_lemmatized = nlp_pipe.clean_text
dictionary = create_dictionary(data_lemmatized)
</code>

### (Standard) LDA model after tuning

In [6]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=term_doc,
                                           id2word=dictionary,
                                           num_topics= 6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=2000,
                                           passes=10,
                                           alpha= 1.5,
                                           per_word_topics=True)

#### Visualize Standard LDA model

In [6]:
# vis_data = pyLDAvis.gensim.prepare(lda_model, term_doc, dictionary, sort_topics=False)
# pyLDAvis.save_html(vis_data, 'std_lda_vis/std_lda_topics=6&a=1.5&batchsize=1.html')

C:\Users\enlik\Miniconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


*With chunksize = 100 after tuning, although the coherence scores are higher than stochastic one('update_every=1'), the topics are not much distinguishable and uninterpretable.*

In [7]:
# lda_model_100 = gensim.models.ldamodel.LdaModel(corpus=term_doc,
#                                            id2word=dictionary,
#                                            num_topics= 15, 
#                                            random_state=100,
#                                            update_every=100,
#                                            chunksize=2000,
#                                            passes=10,
#                                            alpha= 1.5,
#                                            per_word_topics=True)

# vis_data = pyLDAvis.gensim.prepare(lda_model_100, term_doc, dictionary, sort_topics=False)
# pyLDAvis.save_html(vis_data, 'std_lda_vis/std_lda_topics=15&a=1.5&batchsize=100.html')

In [8]:
# coherence_model = CoherenceModel(model=lda_model_100, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')
# np.mean(coherence_model.get_coherence_per_topic())

-0.026390592113391367

## <mark>Mallet's LDA model after tuning &#8592; Best Model
*The difference between Mallet and Gensim’s standard LDA is that Gensim uses a Variational Bayes sampling method which is faster but less precise that Mallet’s Gibbs Sampling.*  [link](https://towardsdatascience.com/basic-nlp-on-the-texts-of-harry-potter-topic-modeling-with-latent-dirichlet-allocation-f3c00f77b0f5)

In [4]:
# mallet_path = '../mallet-2.0.8/bin/mallet' # update this path

import os
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})

# mallet_path = 'mallet-2.0.8/bin/mallet.bat' # update this path
mallet_path = 'C:/mallet-2.0.8/bin/mallet.bat'

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, 
                                             corpus = term_doc,
                                             num_topics = 6, 
                                             random_seed = 100,
                                             id2word = dictionary,
                                             alpha = 1.5)

In [15]:
# ldamallet.save(datapath("model"))
# ldamallet = gensim.models.wrappers.LdaMallet.load(datapath("model"))

#### Visualize Mallet's LDA model

In [16]:
# Visualize the topics
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
pyLDAvis.enable_notebook()

In [17]:
vis = pyLDAvis.gensim.prepare(model, term_doc, dictionary)
pyLDAvis.save_html(vis, 'mallet_lda_vis/mallet_lda_topics=6&a=1.5.html')

C:\Users\enlik\Miniconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [18]:
coherence_model_m = CoherenceModel(model=model, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')
coherence_model_m.get_coherence_per_topic()

[0.025636387283576208,
 -0.00033672387712726484,
 0.027459863226146547,
 0.0016659080521791824,
 -0.06243695078854283,
 0.022359884043397654]

In [19]:
# model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
# model.top_topics(corpus = term_doc,topn=10)

#### For each topic, we could look at frequent and relevant words

In [20]:
frequencies = ldamallet.word_topics.sum(axis=0)
p_word = frequencies / ldamallet.word_topics.sum()
p_word_given_topic = ldamallet.word_topics / np.sum(ldamallet.word_topics, axis = 0)

lamda = 0.5
relevance = lamda * p_word_given_topic + (1-lamda) * p_word_given_topic / p_word 

C:\Users\enlik\AppData\Local\Temp/ipykernel_14508/1698140663.py:3: RuntimeWarning: invalid value encountered in true_divide
  p_word_given_topic = ldamallet.word_topics / np.sum(ldamallet.word_topics, axis = 0)


In [21]:
# topic_dict = {0.: "Platform/Device", 1.: "User Experience", 2.: "Value", 3.: "Service", 4.: "Trouble-shooting", 5.:"Shows"}

# for topic_id in range(6):
#     words = []
#     for id in np.argsort(relevance[topic_id,])[::-1][:15]:
#         words.append(dictionary[id])
#     print(f"Topic: {topic_dict[topic_id]}")
#     print(words)
#     print('\n')

In [22]:
np.argsort(relevance[0,])[::-1][:15]


array([1826, 1540, 4588,  270, 4624, 2459,  279, 5575, 2537,  306, 1493,
       7027,  635,  321, 4734], dtype=int64)

In [23]:
dictionary[1826]

'saw'

In [24]:
topic_dict = {0.: "Platform/Device", 1.: "User Experience", 2.: "Value", 3.: "Service", 4.: "Trouble-shooting", 5.:"Shows"}

for topic_id in range(6):
    words = []
    for id in np.argsort(relevance[topic_id,])[::-1][:15]:
        words.append(dictionary[id])
    print(f"Topic: {topic_dict[topic_id]}")
    print(words)
    print('\n')

Topic: Platform/Device
['saw', 'think', 'around', 'thank', 'out', 'everyone', 'well', 'must', 'wonder', 'self', 'course', 'sometime', 'value', 'up', 'allow']


Topic: User Experience
['think', 'r', 'value', 'well', 'brief', 'noone', 'known', 'more', 'welcome', 'something', 'edu', 'thank', 'down', 'everyone', 'going']


Topic: Value
['going', 'com', 'everyone', 'try', 'twice', 'nothing', 'tell', 'become', 'want', 'see', 'go', 'everything', 'over', 'willing', 'do']


Topic: Service
['out', 'consider', 'up', 'another', 'again', 'around', 'come', 'sub', 'sorry', 'self', 'anyway', 'plus', 'wonder', 'looking', 'allow']


Topic: Trouble-shooting
['sometime', 'thanx', 'more', 'last', 'th', 'around', 'brief', 'nothing', 'as', 'again', 'here', 'up', 'furthermore', 'well', 'thank']


Topic: Shows
['second', 'allow', 'wish', 'take', 'etc', 'on', 'everyone', 'indicate', 'away', 'would', 'anyway', 'took', 'more', 'brief', 'like']




## Interpret the topic model
1. Finding the dominant topic in each document
2. Find the most representative document for each topic
3. Topic distribution across documents
**The code used here for interpretation of the model are based on this website with a little modification by the user myself: <br>
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling**

### Standard LDA - Interpret the model

In [7]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=term_doc, texts=data_lemmatized, df=df)

Getting main topic for document...
0  1000  2000  3000  4000  5000  6000  7000  8000  9000  10000  11000  12000  13000  14000  15000  16000  17000  18000  19000  20000  21000  22000  23000  24000  25000  26000  27000  28000  29000  

#### 1. Standard LDA - Finding the dominant topic in each document

In [8]:
df_dominant_topic = find_dominant_topic_in_each_doc(df_topic_sents_keywords, df=df)
print("Finding the dominant topic in each document")
df_dominant_topic.head(5).style.set_properties(subset=['review'], **{'width': '600px'})

Finding the dominant topic in each document


,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review
0,5.000000,0.377500,"trip, customer, money, charge, issue, company, payment, phone, pay, card","['driver', 'support', 'computer', 'response', 'case', 'response', 'support', 'discount', 'account', 'thought', 'client']",3,"Your drivers are great BUT your support is no good. Why can't it be like Uber where you can actually speak to someone. Instead of getting computer generated responses or like in my case, no response at all. The three stars are for the support. Also I have received a single discount to this account since I installed it three months ago. I'm very disappointed in Bolt, I thought they were there for their clients."
1,4.000000,0.342600,"app, people, fare, rate, passenger, book, travel, request, vehicle, drive","['rating', 'moment', 'driver', 'rating', 'trip', 'driver', 'apartment', 'ride', 'drive', 'destination']",3,"Loving the app, but can only give it a medium rating at the moment as the drivers you give a bad rating on still gets your trip. I had a bad experience with a driver outside my apartment, but he always gets my rides as he's parked there when with my one star and bad review (refused to drive to the destination, rude and aggressive)."
2,5.000000,0.500000,"trip, customer, money, charge, issue, company, payment, phone, pay, card","['response', 'reimbursement', 'part', 'customer', 'promo', 'part', 'situation', 'part', 'company', 'extra_money', 'people', 'card', 'dodgy', 'stick', 'amount', 'card']",4,The response time with regards to customer service was great and thank you. Kindly improve on the reimbursement part because it's not every customer who'll appreciate a promo. That part put some of us in a very sticky situation financially especially now that we have COVID-19. The last part would be for your company not to always take extra money just because people are paying with cards. That's dodgy. Stick to the same amount whether I'm paying cash or with a card.
3,1.000000,0.375000,"time, ride, price, destination, route, order, thing, cost, user, point","['time', 'show', 'estimate', 'amount', 'destination', 'location', 'price', 'feedback', 'uninstall', 'rubbish', 'app']",1,This has been happening for some time now and i always keep quiet. You will show me an estimate of the amount for the destination and when i get to my location the price i will see will be way higher than what you showed. I will stop using you. Just decided to give this feedback before i uninstall this rubbish app
4,3.000000,0.363100,"location, option, way, work, experience, application, update, call, thank, map","['driver', 'option', 'change', 'time', 'transfer', 'trust', 'card', 'cash', 'please_fix', 'problem', 'way']",3,"The drivers almost always never have change and I'm left with no option but to leave my 100-300 naira change with them which I obviously don't intend to. It's not all the time we're able to transfer and not everyone trust using their cards with the app, for those of us that use cash please fix this problem whatever way, it's really annoying."


#### 2. Standard LDA - Find the most representative document for each topic

In [9]:
print("Find the most representative document for each topic")
sent_topics_sorteddf = find_most_representative_doc_for_each_doc(df_topic_sents_keywords,df=df)
sent_topics_sorteddf.style.set_properties(subset=['review'], **{'width': '1000px'})

Find the most representative document for each topic


,Dominant_Topic,Perc_Contribution,Topic_Keywords,star_rating,review
0,0.000000,0.553800,"service, car, taxi, cab, use, day, auto, hour, lot, number",5,I came into NYC this past weekend for a birthday celebration! It was bitter cold and if it wasn't for the via car service the day would not of been pleasant! Both drivers that we had were pleasant and professional. One car was a Camry hybrid and all four of us were very comfortable. The Suburban was our second car and that is a land yacht. I am officially a Via convert!! I really had a great experience!
1,1.000000,0.575300,"time, ride, price, destination, route, order, thing, cost, user, point",3,"Bolt may be cheaper than Uber, but 1) the ETA before the driver arrives is a lie every time, it'll show the driver as 1 min away when in reality it takes 5+ mins, and 2) it takes you on ridiculous detours / down back streets which adds to the time and cost - example: we were a few mins away from our destination in King's Cross and instead, we were taken on a 20+ minute detour through Hampstead/Highgate in North London. No road blocks or anything, just taken on a huge detour for fun I guess."
2,2.000000,0.636100,"driver, minute, love, place, taxi, today, show, min, road, job",3,"I use uber a lot, but lately have been put off as many drivers use heavy aerosol sprays and air- fresheners to disguise smells or simply spruce up the interior. Whilst its welcoming to be in a fresh clean vehicle, the use of these aerosols or air fresheners is bad news for people with airborne allergies and asthma. These scents cause irritation and headaches in sensitive people (like myself and my child who has allergies) Please Uber, educate your members and discourage the use of heavy air scenting- its no different than outdoor air pollution, only we the passengers are subjected to it, indoors whilst also paying for cleaner air fees! The irony! I am sure drivers can maintain cleanliness without resorting to disguise or OTT air scenting. Otherwise Uber is a breath of fresh air in the taxi world. Thanks!"
3,3.000000,0.574800,"location, option, way, work, experience, application, update, call, thank, map",1,"Compared to Clever Taxi, this application is absolutely rubbish! There's no option for me to save the various adresses that I often call taxi at, like I used to do so easily on Clever!! And the localisation is ALWAYS wrong!! It takes ages to get the right location, when I write the correct street name, the application switches it instantly to a wrong street name in the vecinity. Horrendous!!"
4,4.000000,0.569700,"app, people, fare, rate, passenger, book, travel, request, vehicle, drive",3,"There are few problems with this app and there are few suggestions I will make and I hope you act accordingly. When you rate a driver and they don't rate you back, your rate still shows on their profile. I don't think this is either fair or right from your part. I believe there should be an option to change your rate after you see the other persons rate. I always rate people truthfully, but it happened to me a day ago that I got a rate that was subjective, not objective. Apparently I didn't sit well with the guy. I texted him the size of my bag, when I came the car couldn't open the back door. Also the thing for the speed didn't work so he was practically driving by feeling which I didn't like at all. It made me feel very insecure. And plus I was in the front and the firsy person to die when there is a car crash it's not the driver but the person next to him/her. I reported him here on Bla Bla Car but there is no choice to put for a vehicle that isn't functional. I didn't comment on that because I didn't want to upset the other passangers and I was let with a three hour drive that was very unsecure and caused me extreme emotional distress. I left a good grade because everything was fine, the drive, etc. But on second thought I should have put a different grade and warn others, but now the app doesn't allow me. I sent a requ

#### 3. Standard LDA - Topic distribution across documents

In [10]:
# Show
print("Topic distribution across documents")
df_dominant_topic = topic_distribution_across_docs(df_topic_sents_keywords)
df_dominant_topic.style.set_properties(subset=['Keywords'], **{'width': '400px'})

Topic distribution across documents


,Dominant_Topic,Keywords,Num_Documents,Perc_Documents
0,0.000000,"service, car, taxi, cab, use, day, auto, hour, lot, number",7868,0.263900
1,1.000000,"time, ride, price, destination, route, order, thing, cost, user, point",4632,0.155400
2,2.000000,"driver, minute, love, place, taxi, today, show, min, road, job",4586,0.153800
3,3.000000,"location, option, way, work, experience, application, update, call, thank, map",4310,0.144600
4,4.000000,"app, people, fare, rate, passenger, book, travel, request, vehicle, drive",4287,0.143800
5,5.000000,"trip, customer, money, charge, issue, company, payment, phone, pay, card",4130,0.138500


In [11]:
coherence_model = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')
coherence_model.get_coherence_per_topic()

[-0.08499127887857232,
 -0.04825007465208696,
 -0.08316108524893653,
 -0.00571317760274162,
 -0.048308270486805964,
 0.05511379503712159]

### <mark>Mallet Model - Interpret the model</mark>

In [7]:
# ldamallet = gensim.models.wrappers.LdaMallet.load(datapath('model'))
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

In [8]:
df_topic_sents_keywords_m = format_topics_sentences(model, term_doc, data_lemmatized, df)

Getting main topic for document...
0  1000  2000  3000  4000  5000  6000  7000  8000  9000  10000  11000  12000  13000  14000  15000  16000  17000  18000  19000  20000  21000  22000  23000  24000  25000  26000  27000  28000  29000  

In [9]:
all_topics = model.get_document_topics(bow = term_doc)

In [24]:
# df_topic_sents_keywords_m[df_topic_sents_keywords_m[0]]
df_topic_sents_keywords_m

,Dominant_Topic,Perc_Contribution,Keywords,Text
0,2.0,0.7559,"driver, app, ride, trip, service, money, charg...","[driver, support, computer, response, case, re..."
1,1.0,0.6084,"driver, car, service, passenger, time, app, cu...","[rating, moment, driver, rating, trip, driver,..."
2,2.0,0.9262,"driver, app, ride, trip, service, money, charg...","[response, reimbursement, part, customer, prom..."
3,0.0,0.4061,"app, location, ride, option, update, time, wor...","[time, show, estimate, amount, destination, lo..."
4,2.0,0.6228,"driver, app, ride, trip, service, money, charg...","[driver, option, change, time, transfer, trust..."
...,...,...,...,...
29808,3.0,0.7760,"driver, time, taxi, app, car, price, minute, s...","[difficult_get, taxi, time, takes_lot, time, t..."
29809,2.0,0.6183,"driver, app, ride, trip, service, money, charg...","[money, card, money, taxi]"
29810,0.0,0.8025,"app, location, ride, option, update, time, wor...","[ride, extension, apple, map, apple_pay]"
29811,5.0,0.4135,"driver, ride, time, service, cab, location, ap...","[driver, destination, destination, mad]"


In [25]:
# save to pickle file - df_topic_sents_keywords_m
import pickle

output_path = 'lda_mallet_model/'
with open(output_path + 'mallet_lda_df.pkl', 'wb') as f:
    pickle.dump(df_topic_sents_keywords_m, f)


In [ ]:
# save to pickle file - all_reviews_p3.csv
import pickle

output_path = 'raw_data/'
with open(output_path + 'all_reviews_p3.pkl', 'wb') as f:
    pickle.dump(df, f)

#### 1. Mallet LDA - Finding the dominant topic in each document

In [11]:
df_dominant_topic_m = find_dominant_topic_in_each_doc(df_topic_sents_keywords_m, df)
print("Finding the dominant topic in each document")
df_dominant_topic_m.head(5).style.set_properties(subset=['review'], **{'width': '400px'})

Finding the dominant topic in each document


,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review
0,2.000000,0.755900,"driver, app, ride, trip, service, money, charge, amount, card, payment","['driver', 'support', 'computer', 'response', 'case', 'response', 'support', 'discount', 'account', 'thought', 'client']",3,"Your drivers are great BUT your support is no good. Why can't it be like Uber where you can actually speak to someone. Instead of getting computer generated responses or like in my case, no response at all. The three stars are for the support. Also I have received a single discount to this account since I installed it three months ago. I'm very disappointed in Bolt, I thought they were there for their clients."
1,1.000000,0.608400,"driver, car, service, passenger, time, app, customer, rider, trip, route","['rating', 'moment', 'driver', 'rating', 'trip', 'driver', 'apartment', 'ride', 'drive', 'destination']",3,"Loving the app, but can only give it a medium rating at the moment as the drivers you give a bad rating on still gets your trip. I had a bad experience with a driver outside my apartment, but he always gets my rides as he's parked there when with my one star and bad review (refused to drive to the destination, rude and aggressive)."
2,2.000000,0.926200,"driver, app, ride, trip, service, money, charge, amount, card, payment","['response', 'reimbursement', 'part', 'customer', 'promo', 'part', 'situation', 'part', 'company', 'extra_money', 'people', 'card', 'dodgy', 'stick', 'amount', 'card']",4,The response time with regards to customer service was great and thank you. Kindly improve on the reimbursement part because it's not every customer who'll appreciate a promo. That part put some of us in a very sticky situation financially especially now that we have COVID-19. The last part would be for your company not to always take extra money just because people are paying with cards. That's dodgy. Stick to the same amount whether I'm paying cash or with a card.
3,0.000000,0.406100,"app, location, ride, option, update, time, work, issue, phone, number","['time', 'show', 'estimate', 'amount', 'destination', 'location', 'price', 'feedback', 'uninstall', 'rubbish', 'app']",1,This has been happening for some time now and i always keep quiet. You will show me an estimate of the amount for the destination and when i get to my location the price i will see will be way higher than what you showed. I will stop using you. Just decided to give this feedback before i uninstall this rubbish app
4,2.000000,0.622800,"driver, app, ride, trip, service, money, charge, amount, card, payment","['driver', 'option', 'change', 'time', 'transfer', 'trust', 'card', 'cash', 'please_fix', 'problem', 'way']",3,"The drivers almost always never have change and I'm left with no option but to leave my 100-300 naira change with them which I obviously don't intend to. It's not all the time we're able to transfer and not everyone trust using their cards with the app, for those of us that use cash please fix this problem whatever way, it's really annoying."


In [12]:
df_dominant_topic_m

,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review
0,2.0,0.7559,"driver, app, ride, trip, service, money, charg...","[driver, support, computer, response, case, re...",3,Your drivers are great BUT your support is no ...
1,1.0,0.6084,"driver, car, service, passenger, time, app, cu...","[rating, moment, driver, rating, trip, driver,...",3,"Loving the app, but can only give it a medium ..."
2,2.0,0.9262,"driver, app, ride, trip, service, money, charg...","[response, reimbursement, part, customer, prom...",4,The response time with regards to customer ser...
3,0.0,0.4061,"app, location, ride, option, update, time, wor...","[time, show, estimate, amount, destination, lo...",1,This has been happening for some time now and ...
4,2.0,0.6228,"driver, app, ride, trip, service, money, charg...","[driver, option, change, time, transfer, trust...",3,The drivers almost always never have change an...
...,...,...,...,...,...,...
29808,3.0,0.7760,"driver, time, taxi, app, car, price, minute, s...","[difficult_get, taxi, time, takes_lot, time, t...",3,It’s rather difficult to get a taxi. Sometimes...
29809,2.0,0.6183,"driver, app, ride, trip, service, money, charg...","[money, card, money, taxi]",1,Taxi drivers don’t speak Estonian or English. ...
29810,0.0,0.8025,"app, location, ride, option, update, time, wor...","[ride, extension, apple, map, apple_pay]",3,Ride booking extension in Apple maps not avail...
29811,5.0,0.4135,"driver, ride, time, service, cab, location, ap...","[driver, destination, destination, mad]",1,"I ordered a taxi, but when it arrived the driv..."


#### 2. Mallet LDA - Find the most representative document for each topic

In [32]:
print("Find the most representative document for each topic")
sent_topics_sorteddf_m = find_most_representative_doc_for_each_doc(df_topic_sents_keywords_m, df)
sent_topics_sorteddf_m.style.set_properties(subset=['review'], **{'width': '400px'})

Find the most representative document for each topic


,Dominant_Topic,Perc_Contribution,Topic_Keywords,star_rating,review
0,0.000000,0.928000,"app, location, ride, option, update, time, work, issue, phone, number",1,Frustrated by such data hogging apps. My Personal Data is compromised by this app as it makes granting access to my sms mandatory. This permission is not atall essential for the app function and is a clear attempt to extract personal and financial information from my messages. App store should ban this app till such data violations are plugged. Uninstalling the app immediately.
1,1.000000,0.936800,"driver, car, service, passenger, time, app, customer, rider, trip, route",3,"The app is easy to use and convenient. However, it favours drivers over passengers. For example, if a passenger cancels a trip, he/she will be charged £5 in London. But drivers pick up and cancel trips all the time. Several times, I’ve been in a situation, when 3 or 4 drivers in a row picked up my trip, I waited for 7-10 minutes for each of them, and then they cancelled (I ended up being late). There was no bad traffic in the area. For me as a customer, it feels really unfair. Also, the ratings system is confusing. Drivers cannot see their ratings, customers cannot see the ratings before they order. Ratings here are pointless."
2,2.000000,0.968400,"driver, app, ride, trip, service, money, charge, amount, card, payment",1,"One sudden day, my ola account got blocked for permanently. When contacted the customer care, they said they had no information about my account getting blocked and asked me to drop mail to the ola grievance team, ever since the last 2 months, I have mailed multiple time s to the grievance team and other ola support emails, but got no valid response although have kept on receiving multiple mails and calls, for the payment of due ola money charges throughout last two months, I have confirmed them over the email and even told the customer care agent who called asking to pay the dues that, I would be able to pay the charges only if my account is unblocked. Although account is still blocked, I received a mail on 6th october stating that, as I have failed to pay the dues for around last 71 days, they would take legal action against me, ever since I received this mail, I have sent several mails to ola grievance and other support teams, asking to unblock my account first, thereafter I would clear the dues, as I have never denied to clear the dues, but till now, not received any valid response from them. They are least bothered. Have tried contacting the contact centres over phone, but each time the call is disconnected after initially getting connected to ola customer care, they have cut down all the channels to reach out to them over the phone. They have the worst possible customer service. Pathetic...!!!"
3,3.000000,0.930600,"driver, time, taxi, app, car, price, minute, service, work, trip",3,"The prices vary multiple euros (more expensive) if the taxi is using a taximeter or using the app for the fare pricing. The app calculates a higher fare charge if you take same taxi fare pricing, but with different means of measurments. The trip from work to home costs around 7-8 euros with a taximeter taxi, but the private or the a taxi what uses the app, its 10-11 euros, even though the pricing is sometimes even cheaper than the taximeter taxi."
4,4.000000,0.905700,"service, love, ride, driver, app, price, time, travel, experience, discount",5,"A great way to share car travel, while you make best use of fuel and the driving efforts, get a good company to chat along. This App is helping save thousands of tonnes of carbon emissions."
5,5.000000,0.937200,"driver, ride, time, service, cab, location, app, auto, trip, customer",1,"I almost missed my train! I'd pre booked a ride from Gandhinagar to Ahmedabad railway station on 6-6- 2019. I got a msg that the ride was scheduled & the details would be msgd to me 10 mins prior the pickup time. The scheduled ride got cancelled without any intimation/msg d

#### 3. Mallet LDA - Topic distribution across documents

In [33]:
# Show
print("Topic distribution across documents")
df_dominant_topic_m = topic_distribution_across_docs(df_topic_sents_keywords_m)
df_dominant_topic_m.style.set_properties(subset=['Keywords'], **{'width': '400px'})

Topic distribution across documents


,Dominant_Topic,Keywords,Num_Documents,Perc_Documents
0,0.000000,"app, location, ride, option, update, time, work, issue, phone, number",7520,0.252200
1,1.000000,"driver, car, service, passenger, time, app, customer, rider, trip, route",5770,0.193500
2,2.000000,"driver, app, ride, trip, service, money, charge, amount, card, payment",4943,0.165800
3,3.000000,"driver, time, taxi, app, car, price, minute, service, work, trip",4342,0.145600
4,4.000000,"service, love, ride, driver, app, price, time, travel, experience, discount",3649,0.122400
5,5.000000,"driver, ride, time, service, cab, location, app, auto, trip, customer",3589,0.120400


In [34]:
coherence_model = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')
coherence_model.get_coherence_per_topic()

[0.025636387283576208,
 -0.00033672387712726484,
 0.027459863226146547,
 0.0016659080521791824,
 -0.06243695078854283,
 0.022359884043397654]

## Try the model with TF-IDF dataset

### Standard LDA with TF-IDF

In [35]:
lda_model_tfidf = gensim.models.ldamodel.LdaModel(corpus=tf_idf,
                                           num_topics= 6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=2000,
                                           passes=10,
                                           alpha=1.5,
                                           per_word_topics=True)

In [36]:
vis_data = pyLDAvis.gensim.prepare(lda_model_tfidf, tf_idf, dictionary, sort_topics=False)
pyLDAvis.save_html(vis_data, 'std_lda_vis/std_lda_vis_tfidf_num_topics=6&alpha=1.5.html')

C:\Users\enlik\Miniconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [37]:
coherence_model_m = CoherenceModel(model=lda_model_tfidf, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')
coherence_model_m.get_coherence_per_topic()

[-0.0396091223390339,
 -0.060167577309842894,
 -0.023276531233836457,
 -0.10655968548248082,
 -0.04118598388871035,
 0.023076828225845062]

In [39]:
df_topic_sents_keywords_tfidf = format_topics_sentences(ldamodel=lda_model_tfidf, corpus=term_doc, texts=data_lemmatized, df=df)

Getting main topic for document...
0  1000  2000  3000  4000  5000  6000  7000  8000  9000  10000  11000  12000  13000  14000  15000  16000  17000  18000  19000  20000  21000  22000  23000  24000  25000  26000  27000  28000  29000  

#### 1. Standard LDA with TF-IDF - Finding the dominant topic in each document

In [40]:
df_dominant_topic_tfidf = find_dominant_topic_in_each_doc(df_topic_sents_keywords_tfidf, df)
print("Finding the dominant topic in each document")
df_dominant_topic_tfidf.head(5).style.set_properties(subset=['review'], **{'width': '400px'})

Finding the dominant topic in each document


,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review
0,5.000000,0.368100,"135, 427, 18, 160, 17, 0, 19, 37, 104, 593","['driver', 'support', 'computer', 'response', 'case', 'response', 'support', 'discount', 'account', 'thought', 'client']",3,"Your drivers are great BUT your support is no good. Why can't it be like Uber where you can actually speak to someone. Instead of getting computer generated responses or like in my case, no response at all. The three stars are for the support. Also I have received a single discount to this account since I installed it three months ago. I'm very disappointed in Bolt, I thought they were there for their clients."
1,4.000000,0.381700,"28, 5, 23, 59, 63, 95, 237, 15, 205, 519","['rating', 'moment', 'driver', 'rating', 'trip', 'driver', 'apartment', 'ride', 'drive', 'destination']",3,"Loving the app, but can only give it a medium rating at the moment as the drivers you give a bad rating on still gets your trip. I had a bad experience with a driver outside my apartment, but he always gets my rides as he's parked there when with my one star and bad review (refused to drive to the destination, rude and aggressive)."
2,5.000000,0.449800,"135, 427, 18, 160, 17, 0, 19, 37, 104, 593","['response', 'reimbursement', 'part', 'customer', 'promo', 'part', 'situation', 'part', 'company', 'extra_money', 'people', 'card', 'dodgy', 'stick', 'amount', 'card']",4,The response time with regards to customer service was great and thank you. Kindly improve on the reimbursement part because it's not every customer who'll appreciate a promo. That part put some of us in a very sticky situation financially especially now that we have COVID-19. The last part would be for your company not to always take extra money just because people are paying with cards. That's dodgy. Stick to the same amount whether I'm paying cash or with a card.
3,3.000000,0.249300,"31, 39, 111, 124, 270, 276, 44, 147, 317, 290","['time', 'show', 'estimate', 'amount', 'destination', 'location', 'price', 'feedback', 'uninstall', 'rubbish', 'app']",1,This has been happening for some time now and i always keep quiet. You will show me an estimate of the amount for the destination and when i get to my location the price i will see will be way higher than what you showed. I will stop using you. Just decided to give this feedback before i uninstall this rubbish app
4,3.000000,0.266900,"31, 39, 111, 124, 270, 276, 44, 147, 317, 290","['driver', 'option', 'change', 'time', 'transfer', 'trust', 'card', 'cash', 'please_fix', 'problem', 'way']",3,"The drivers almost always never have change and I'm left with no option but to leave my 100-300 naira change with them which I obviously don't intend to. It's not all the time we're able to transfer and not everyone trust using their cards with the app, for those of us that use cash please fix this problem whatever way, it's really annoying."


#### 2. Standard LDA with TF-IDF - Find the most representative document for each topic

In [41]:
print("Find the most representative document for each topic")
sent_topics_sorteddf_tfidf = find_most_representative_doc_for_each_doc(df_topic_sents_keywords_tfidf, df)
sent_topics_sorteddf_tfidf.style.set_properties(subset=['review'], **{'width': '400px'})

Find the most representative document for each topic


,Dominant_Topic,Perc_Contribution,Topic_Keywords,star_rating,review
0,0.000000,0.532800,"45, 155, 529, 546, 84, 745, 415, 368, 15, 255",5,"Amazing great idea and service. I am on business trip in Spain and started using ""normal"" taxi service. Overpriced, expensive, driving around the city instead of straight to the destination point. After one week started using Cabify service. Which is simply, easy, cheap and transparent. I would recommend to everyone. Greg."
1,1.000000,0.558800,"14, 32, 121, 35, 10, 224, 249, 28, 453, 438",5,"Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it,"
2,2.000000,0.517700,"5, 35, 172, 61, 364, 137, 41, 529, 96, 34",2,"This was way better earlier. Last 4 times we called for Bolt: estimate time was showing 2-4 minutes. Arrival times were 10-15. When they show where driver is on map, it is obvious, that driver cannot make it even in 10 minutes, but app insists it will arrive in 2 mintues for the remaining time of 10 minutes. If I see Bolt that it will come after 15 minutes, I would cancel that driver and look for another one, however I will get penalized for that. So either stop lying about estimated arrival times, or stop penalizing for 10 minutes late drivers, when there is another one nearby."
3,3.000000,0.558700,"31, 39, 111, 124, 270, 276, 44, 147, 317, 290",3,"When I go to input my car details 'Sport' is missing on comfort and 'coupe' is missing in vehicle type. A sound system category would be good, this is important for long trips, I have a 1000w sound system and would like to let them know if that interests them at all. How about just allow people an open text description box called 'about my car'"
4,4.000000,0.545300,"28, 5, 23, 59, 63, 95, 237, 15, 205, 519",1,"People want something like Uber, but since the taxi drivers in Ireland are terrified they won't be able to rip people off anymore they refuse to allow it and have made this as similar as possible, but made sure they can still rip us off.. It's as simple as letting us know how much our journey will cost us before we order the taxi.. It should always give the taxi driver our actual location, because most people using this is drunk so they won't actually move the marker manually. The Taxi drivers will tell you themselves that people are always at the wrong location. Uber is better. To bad we're in Ireland."
5,5.000000,0.581500,"135, 427, 18, 160, 17, 0, 19, 37, 104, 593",1,I have been charged twice and still.. Im getting call multiple times. As postpaid amount is due.. Where i have not used any. And paid via Phonepay.. Many times sent mail with transaction proof. Till now no replies and customer pathetic never receives call.. Still everyday notifications of due amount.


#### 3. Standard LDA with TF-IDF - Topic distribution across documents

In [42]:
# Show
print("Topic distribution across documents")
df_dominant_topic_tfidf = topic_distribution_across_docs(df_topic_sents_keywords_tfidf)
df_dominant_topic_tfidf.style.set_properties(subset=['Keywords'], **{'width': '400px'})

Topic distribution across documents


,Dominant_Topic,Keywords,Num_Documents,Perc_Documents
0,0.000000,"45, 155, 529, 546, 84, 745, 415, 368, 15, 255",7440,0.249600
1,1.000000,"14, 32, 121, 35, 10, 224, 249, 28, 453, 438",5800,0.194500
2,2.000000,"5, 35, 172, 61, 364, 137, 41, 529, 96, 34",4466,0.149800
3,3.000000,"31, 39, 111, 124, 270, 276, 44, 147, 317, 290",4449,0.149200
4,4.000000,"28, 5, 23, 59, 63, 95, 237, 15, 205, 519",3841,0.128800
5,5.000000,"135, 427, 18, 160, 17, 0, 19, 37, 104, 593",3817,0.128000


In [43]:
coherence_model_tfidf = CoherenceModel(model=lda_model_tfidf, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')

In [44]:
coherence_model_tfidf.get_coherence_per_topic()

[-0.0396091223390339,
 -0.060167577309842894,
 -0.023276531233836457,
 -0.10655968548248082,
 -0.04118598388871035,
 0.023076828225845062]

### Mallet's LDA with TF-IDF

In [45]:
import os
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})

mallet_path = 'C:/mallet-2.0.8/bin/mallet.bat'

ldamallet_tfidf = gensim.models.wrappers.LdaMallet(mallet_path, 
                                                   corpus=tf_idf, 
                                                   num_topics=6, 
                                                   random_seed = 100,
                                                   id2word=dictionary,
                                                   alpha = 1.5)

In [46]:
# Visualize the topics
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet_tfidf)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, tf_idf, dictionary)
pyLDAvis.save_html(vis, 'mallet_lda_vis/mallet_lda_vis_tfidf_num_topics=6&alpha=1.5.html')

In [47]:
coherence_model_m = CoherenceModel(model=model, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')
coherence_model_m.get_coherence_per_topic()

[-0.34257277514201,
 -0.30710748357391965,
 -0.1448369798718913,
 -0.30519371034115916,
 -0.2651728781887421,
 -0.2874400049962764]

In [49]:
df_topic_sents_keywords_tfidf_m = format_topics_sentences(ldamodel=ldamallet_tfidf, corpus=term_doc, texts=data_lemmatized, df=df)

Getting main topic for document...
0  1000  2000  3000  4000  5000  6000  7000  8000  9000  10000  11000  12000  13000  14000  15000  16000  17000  18000  19000  20000  21000  22000  23000  24000  25000  26000  27000  28000  29000  

#### 1. Mallet's LDA with TF-IDF - Finding the dominant topic in each document

In [50]:
df_dominant_topic_tfidf_m = find_dominant_topic_in_each_doc(df_topic_sents_keywords_tfidf_m, df)
print("Finding the dominant topic in each document")
df_dominant_topic_tfidf_m.head(5).style.set_properties(subset=['review'], **{'width': '400px'})

Finding the dominant topic in each document


,Dominant_Topic,Perc_Contribution,Keywords,Text,rating,review
0,3.000000,0.320100,"service, experience, app, friendly_driver, good, cab, good_price, good_application, star, polite_driver","['driver', 'support', 'computer', 'response', 'case', 'response', 'support', 'discount', 'account', 'thought', 'client']",3,"Your drivers are great BUT your support is no good. Why can't it be like Uber where you can actually speak to someone. Instead of getting computer generated responses or like in my case, no response at all. The three stars are for the support. Also I have received a single discount to this account since I installed it three months ago. I'm very disappointed in Bolt, I thought they were there for their clients."
1,2.000000,0.362400,"app, driver, easy_use, discount, guy, transportation, location, promotion, drive, transport","['rating', 'moment', 'driver', 'rating', 'trip', 'driver', 'apartment', 'ride', 'drive', 'destination']",3,"Loving the app, but can only give it a medium rating at the moment as the drivers you give a bad rating on still gets your trip. I had a bad experience with a driver outside my apartment, but he always gets my rides as he's parked there when with my one star and bad review (refused to drive to the destination, rude and aggressive)."
2,3.000000,0.323000,"service, experience, app, friendly_driver, good, cab, good_price, good_application, star, polite_driver","['response', 'reimbursement', 'part', 'customer', 'promo', 'part', 'situation', 'part', 'company', 'extra_money', 'people', 'card', 'dodgy', 'stick', 'amount', 'card']",4,The response time with regards to customer service was great and thank you. Kindly improve on the reimbursement part because it's not every customer who'll appreciate a promo. That part put some of us in a very sticky situation financially especially now that we have COVID-19. The last part would be for your company not to always take extra money just because people are paying with cards. That's dodgy. Stick to the same amount whether I'm paying cash or with a card.
3,1.000000,0.340600,"driver, trip, travel, friendly_driver, promo, quick_efficient, driving, company, recommend, enjoy","['time', 'show', 'estimate', 'amount', 'destination', 'location', 'price', 'feedback', 'uninstall', 'rubbish', 'app']",1,This has been happening for some time now and i always keep quiet. You will show me an estimate of the amount for the destination and when i get to my location the price i will see will be way higher than what you showed. I will stop using you. Just decided to give this feedback before i uninstall this rubbish app
4,5.000000,0.404800,"driver, people, drivers_alway, update, good_service, day, application, code, city, job","['driver', 'option', 'change', 'time', 'transfer', 'trust', 'card', 'cash', 'please_fix', 'problem', 'way']",3,"The drivers almost always never have change and I'm left with no option but to leave my 100-300 naira change with them which I obviously don't intend to. It's not all the time we're able to transfer and not everyone trust using their cards with the app, for those of us that use cash please fix this problem whatever way, it's really annoying."


#### 2. Mallet's LDA with TF-IDF - Find the most representative document for each topic

In [51]:
print("Find the most representative document for each topic")
sent_topics_sorteddf_tfidf_m = find_most_representative_doc_for_each_doc(df_topic_sents_keywords_tfidf_m, df)
sent_topics_sorteddf_tfidf_m.style.set_properties(subset=['review'], **{'width': '400px'})

Find the most representative document for each topic


,Dominant_Topic,Perc_Contribution,Topic_Keywords,star_rating,review
0,0.000000,0.772700,"time, price, work, car, fast_efficient, money, destination, amazing_service, place, reliable_efficient",3,"Most of the times it shows an ETA, let's say 2 minutes, but it keeps changing and end up waiting 15 minutes, but it still displays 2-3 minutes."
1,0.000000,0.772700,"time, price, work, car, fast_efficient, money, destination, amazing_service, place, reliable_efficient",4,"it would be nice if it was possible to choose maybe from 3 cars ( by time ), not always needing a car in 2 minutes...."
2,0.000000,0.772700,"time, price, work, car, fast_efficient, money, destination, amazing_service, place, reliable_efficient",1,"How can you charge 7500 from ikenegbu to Futo and back to Ikenegbu? The time had already started reading 30-35 minutes before I even entered the car. But I didn't say anything... Then while we were coming back we were stopped and searched by the police . All the time they wasted arguing still added to my bills. I'm very disappointed. I am deleting this app now, going back to my URU."
3,1.000000,0.833300,"driver, trip, travel, friendly_driver, promo, quick_efficient, driving, company, recommend, enjoy",1,Auto driver charged extra amount......we booking ola for low price.... but they charged extra amount......we will give always feedback but you not responding..... very bad and worst app.... because of drivers
4,2.000000,0.839900,"app, driver, easy_use, discount, guy, transportation, location, promotion, drive, transport",1,"I already have an Uber account. The app doesn’t give me the option to log into the existing account. The App starts with a screen asking my mobile and when I enter my mobile, e message says this mobile is already registered. Obviously registered because it is my bloody account and I don’t want to create another account. I just want to log into my existing account. How and where???? Useless"
5,3.000000,0.807700,"service, experience, app, friendly_driver, good, cab, good_price, good_application, star, polite_driver",3,Please make available for Kindle Fire Phone. Missing out on a share of the market there! Would of been 5 stars. Get this sorted so I can start using it again!!
6,3.000000,0.807700,"service, experience, app, friendly_driver, good, cab, good_price, good_application, star, polite_driver",1,"Terrible service missed my train, prebooked my cab 2 weeks ago and one the day it neverturned up , details will shown before 10mins of pick up just vanished, I was stuck. Pathetic cab service totally unreliable."
7,4.000000,0.868400,"ride, love, rate, taxi, option, route, customer, fun, rider, map",5,"Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it, Love it,"
8,5.000000,0.772700,"driver, people, drivers_alway, update, good_service, day, application, code, city, job",5,"Cornelius is absolutely the best driver and so kind. Ive had a great experience with all the drivers, but Cornelius is my favorite. Have a blessed day everyone."


#### 3. Mallet's LDA with TF-IDF - Topic distribution across documents

In [52]:
# Show
print("Topic distribution across documents")
df_dominant_topic_tfidf_m = topic_distribution_across_docs(df_topic_sents_keywords_tfidf_m)
df_dominant_topic_tfidf_m.style.set_properties(subset=['Keywords'], **{'width': '400px'})

Topic distribution across documents


,Dominant_Topic,Keywords,Num_Documents,Perc_Documents
0,0.000000,"time, price, work, car, fast_efficient, money, destination, amazing_service, place, reliable_efficient",9025,0.302700
1,1.000000,"driver, trip, travel, friendly_driver, promo, quick_efficient, driving, company, recommend, enjoy",5035,0.168900
2,2.000000,"app, driver, easy_use, discount, guy, transportation, location, promotion, drive, transport",4575,0.153500
3,3.000000,"service, experience, app, friendly_driver, good, cab, good_price, good_application, star, polite_driver",4152,0.139300
4,4.000000,"ride, love, rate, taxi, option, route, customer, fun, rider, map",3955,0.132700
5,5.000000,"driver, people, drivers_alway, update, good_service, day, application, code, city, job",3071,0.103000


In [53]:
coherence_model_tfidf_m = CoherenceModel(model=ldamallet_tfidf, texts=data_lemmatized, dictionary=dictionary, coherence='c_npmi')

In [54]:
coherence_model_tfidf_m.get_coherence_per_topic()

[-0.34257277514201,
 -0.30710748357391965,
 -0.1448369798718913,
 -0.30519371034115916,
 -0.2651728781887421,
 -0.2874400049962764]